In [1]:
print("Hola Mundo")

Hola Mundo


In [1]:
import pandas as pd

In [8]:
df = pd.read_csv("tweets_evento.csv")
df.head()

,id,autor,fecha,texto
0,1947075935557054835,1358564234691551233,2025-07-20 23:27:32+00:00,@derick52230189 @PabloWayne6 @Dusan__Tadic @FC...
1,1947064347341058233,41182147,2025-07-20 22:41:29+00:00,La portera Ann-Katrin Berger se convirtió en f...
2,1947053891570311596,709853460619993088,2025-07-20 21:59:56+00:00,@NeoDulcineo @EmilioDelgadoOr @FelixLopez_Rey ...
3,1947025922843877825,457819234,2025-07-20 20:08:48+00:00,@GilbertoSsj @CarlosFC1001 @TheBigLudwig Lamin...
4,1947025699736285297,457819234,2025-07-20 20:07:55+00:00,@GilbertoSsj @IvanSilva135097 Lamine Yamal a e...


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      100 non-null    int64 
 1   autor   100 non-null    int64 
 2   fecha   100 non-null    object
 3   texto   100 non-null    object
dtypes: int64(2), object(2)
memory usage: 3.3+ KB


In [10]:
pip install praw

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [praw]
Note: you may need to restart the kernel to use updated packages.


In [2]:
import praw

reddit = praw.Reddit(
    client_id='zRF0ZieKs1Q2mLgpb76x7g',
    client_secret='uunPz8KIIhhLq0rQp_QkhPejdBC1FQ',
    user_agent='script:sentimientos-app:v1.0 (by u/According-Sherbet796)',
    username='According-Sherbet796',
    password='Adrian123'
)

# Probar que la autenticación fue exitosa
print("Autenticado como:", reddit.user.me())

Autenticado como: According-Sherbet796


In [18]:

# Escoge un subreddit y un post específico (puedes usar hot, new, top, etc.)
subreddit = reddit.subreddit("askreddit")  # o cualquier otro
for post in subreddit.hot(limit=1):  # Puedes cambiar el límite
    print(f"Título: {post.title}")
    post.comments.replace_more(limit=0)  # Ignora los comentarios colapsados
    for comment in post.comments.list():
        print(comment.body)

DEBUG:prawcore:Fetching: GET https://oauth.reddit.com/r/askreddit/hot at 1753058018.4019961
DEBUG:prawcore:Data: None
DEBUG:prawcore:Params: {'limit': 1, 'raw_json': 1}
DEBUG:urllib3.connectionpool:https://oauth.reddit.com:443 "GET /r/askreddit/hot?limit=1&raw_json=1 HTTP/1.1" 200 1143
DEBUG:prawcore:Response: 200 (1143 bytes) (rst-379:rem-998.0:used-2 ratelimit) at 1753058018.701899
DEBUG:prawcore:Fetching: GET https://oauth.reddit.com/comments/1m4v7lx/ at 1753058018.704745
DEBUG:prawcore:Data: None
DEBUG:prawcore:Params: {'limit': 2048, 'raw_json': 1, 'sort': 'confidence'}


Título: People who’ve attended multiple weddings for the same person (i.e., they got divorced and remarried to someone else) — what differences did you notice between the two weddings?


DEBUG:urllib3.connectionpool:https://oauth.reddit.com:443 "GET /comments/1m4v7lx/?limit=2048&sort=confidence&raw_json=1 HTTP/1.1" 200 124912
DEBUG:prawcore:Response: 200 (124912 bytes) (rst-379:rem-997.0:used-3 ratelimit) at 1753058022.646252


I have an aunt who has been married 7 times. When she called mom to invite her to...number 5, Mom said, "No thanks. I'll catch the next one." 

I think they didn't speak for the duration of that marriage. 

My aunt is a little out there.
My sister has been married four times. First wedding was right out of high school, high school sweetheart deal, kind of an off the cuff event for two dumb teenagers before her new husband went off to become a Navy Seal. 

Second wedding several years later was a shotgun deal, hippie vibes, at a cabin in the woods, with my 6 month old neice playing the part of her parents' flower girl. 

Third wedding was this massive, over-the-top lakeside country club shin dig, like she was trying to prove a point? I barely remember it aside from being exhausted and astounded by the price tag. 

Annnd the fourth wedding? I wouldn't know, they eloped and didn't invite anyone from our side of the family. LOL. I guess fourth time is the charm though, because they're stil

In [20]:
pip install textblob

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 624.3/624.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 6.4 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: nltk
    Found existing installation: nltk 3.8.1
    Uninstalling nltk-3.8.1:
      Successfully uninstalled nltk-3.8.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [textblob]
Note: you may need to restart the kernel to use updated packages.


In [3]:
from textblob import TextBlob

In [4]:
# URL de la publicación (puedes cambiarla)
url = "https://www.reddit.com/r/WatchPeopleDieInside/comments/1j0gln4/ukrainian_ambassador_to_us_when_hearing_trump_and/"
submission = reddit.submission(url=url)

# Cargar todos los comentarios
submission.comments.replace_more(limit=0)  # Cargar todos los comentarios visibles

# Analizar polaridad
for i, comment in enumerate(submission.comments[:20]):  # Puedes ajustar la cantidad de comentarios
    texto = comment.body
    blob = TextBlob(texto)
    polaridad = blob.sentiment.polarity
    print(f"{i+1}. Comentario: {texto[:100]}...")  # muestra solo los primeros 100 caracteres
    print(f"   ➤ Polaridad: {polaridad}\n")

1. Comentario: I cannot imagine having that job at this exact moment......
   ➤ Polaridad: 0.25

2. Comentario: What a horrible day to have a brain...
   ➤ Polaridad: -1.0

3. Comentario: You and me girl...
   ➤ Polaridad: 0.0

4. Comentario: I wanna ask this woman if she had a headache or a downright migraine after witnessing this because I...
   ➤ Polaridad: -0.275

5. Comentario: I had this reaction too. Really she wanted to cry, but she was trying to keep it together....
   ➤ Polaridad: 0.2

6. Comentario: For me, this is much sadder than the argument. She knows the consequences of losing US support. It c...
   ➤ Polaridad: 0.2

7. Comentario: Time to wake up, Europe. History might repeat itself.

edit: I'm European, to be clear, considering ...
   ➤ Polaridad: 0.05000000000000001

8. Comentario: She looks like she's gonna need a lotta Ativan these coming months. I feel for her 😔...
   ➤ Polaridad: 0.0

9. Comentario: Economy crashing, -1.5% GDP growth predicted for Q1, allies aban

In [6]:
import csv

In [7]:
# URL del post
url = "https://www.reddit.com/r/WatchPeopleDieInside/comments/1j0gln4/ukrainian_ambassador_to_us_when_hearing_trump_and/"
submission = reddit.submission(url=url)

# Cargar todos los comentarios visibles
submission.comments.replace_more(limit=0)
comentarios = submission.comments.list()

# Crear y guardar el archivo CSV
with open("comentarios_reddit.csv", mode="w", newline="", encoding="utf-8") as archivo_csv:
    writer = csv.writer(archivo_csv)
    writer.writerow(["Comentario", "Polaridad", "Longitud"])

    for comment in comentarios:
        texto = comment.body
        blob = TextBlob(texto)
        polaridad = blob.sentiment.polarity
        writer.writerow([texto, polaridad, len(texto)])

print("✅ Comentarios guardados en comentarios_reddit.csv")

✅ Comentarios guardados en comentarios_reddit.csv


In [8]:
df = pd.read_csv("comentarios_reddit.csv")
df.head()

,Comentario,Polaridad,Longitud
0,I cannot imagine having that job at this exact...,0.250,56
1,What a horrible day to have a brain,-1.000,35
2,You and me girl,0.000,15
3,I wanna ask this woman if she had a headache o...,-0.275,191
4,I had this reaction too. Really she wanted to ...,0.200,90


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 496 entries, 0 to 495
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Comentario  496 non-null    object 
 1   Polaridad   496 non-null    float64
 2   Longitud    496 non-null    int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 11.8+ KB


In [13]:
from datetime import datetime

In [25]:
# URL de la publicación
url = "https://www.reddit.com/r/WatchPeopleDieInside/comments/1j0gln4/ukrainian_ambassador_to_us_when_hearing_trump_and/"
submission = reddit.submission(url=url)

# Expandir todos los comentarios
submission.comments.replace_more(limit=0)
comentarios = submission.comments.list()  # incluye todos los niveles

datos = []

for comment in comentarios:
    texto = comment.body.strip()
    if not texto:
        continue  # omitir vacíos o eliminados

    autor = comment.author
    if autor:
        nombre = autor.name
        try:
            creado = datetime.utcfromtimestamp(autor.created_utc).strftime('%Y-%m-%d')
        except:
            creado = ""
    else:
        nombre = "[eliminado]"
        creado = ""

    datos.append([nombre, creado, texto])

# Guardar en CSV
with open("comentarios_reddit_con_usuario.csv", mode="w", newline="", encoding="utf-8") as archivo_csv:
    writer = csv.writer(archivo_csv)
    writer.writerow(["Usuario", "Fecha_Creación", "Comentario"])
    writer.writerows(datos)

print(f"✅ Se guardaron {len(datos)} comentarios en 'comentarios_reddit_con_usuario.csv'")

✅ Se guardaron 496 comentarios en 'comentarios_reddit_con_usuario.csv'


In [26]:
df = pd.read_csv("comentarios_reddit_con_usuario.csv")

In [27]:
df.head()

,Usuario,Fecha_Creación,Comentario
0,Healthy_Block3036,2020-11-28,I cannot imagine having that job at this exact...
1,Sad_Advice_8152,NaN,What a horrible day to have a brain
2,chunkykongracing,2021-06-21,You and me girl
3,yurtzwisdomz,2024-06-30,I wanna ask this woman if she had a headache o...
4,ChristineBorus,2019-03-20,I had this reaction too. Really she wanted to ...


In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 496 entries, 0 to 495
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Usuario         496 non-null    object
 1   Fecha_Creación  457 non-null    object
 2   Comentario      496 non-null    object
dtypes: object(3)
memory usage: 11.8+ KB


In [ ]:
import time
time.sleep(1)  # espera entre peticiones